In [1]:
from gurobipy import GRB
import gurobipy as gb

In [3]:
import gurobipy as gb

# Create a new optimization model
model = gb.Model("Sunnyshore Bay Linear Program")

# Decision Variables
X = {}
months = ['May', 'June', 'July', 'August']
term_structures = [1, 2, 3]

for month in months:
    for term in term_structures:
        X[month, term] = model.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name=f"X_{month}_{term}")


# Objective Function: Minimize the total amount repaid to the bank
model.setObjective(
    gb.quicksum((X[month, term] + X[month, term] * interest_rate) for month in months for term, interest_rate in zip(term_structures, [0.0175, 0.0225, 0.0275])),
    gb.GRB.MINIMIZE
)

# Cash balance constraints
model.addConstr(140000 + gb.quicksum(X['May', term] for term in term_structures) >= 25000, "Cash_Balance_May")
model.addConstr(140000 + gb.quicksum(X['May', term] for term in term_structures) + gb.quicksum(X['June', term] for term in term_structures) >= 20000, "Cash_Balance_June")
model.addConstr(140000 + gb.quicksum(X['May', term] for term in term_structures) + gb.quicksum(X['June', term] for term in term_structures) + gb.quicksum(X['July', term] for term in term_structures) >= 35000, "Cash_Balance_July")
model.addConstr(140000 + gb.quicksum(X['May', term] for term in term_structures) + gb.quicksum(X['June', term] for term in term_structures) + gb.quicksum(X['July', term] for term in term_structures) + gb.quicksum(X['August', term] for term in term_structures) >= 18000, "Cash_Balance_August")

# Total borrowed amount constraints
model.addConstr(gb.quicksum(X['May', term] for term in term_structures) <= 250000, "Total_Borrowed_May")
model.addConstr(gb.quicksum(X['June', term] for term in term_structures) <= 150000, "Total_Borrowed_June")
model.addConstr(gb.quicksum(X['July', term] for term in term_structures) <= 350000, "Total_Borrowed_July")

# Cash balance at the end of July constraint
model.addConstr(140000 + gb.quicksum(X['May', term] for term in term_structures) + gb.quicksum(X['June', term] for term in term_structures) + gb.quicksum(X['July', term] for term in term_structures) >= 0.65 * ((140000 + gb.quicksum(X['May', term] for term in term_structures)) + (140000 + gb.quicksum(X['May', term] for term in term_structures) + gb.quicksum(X['June', term] for term in term_structures))), "Cash_Balance_End_July")

# Optimize the model
model.optimize()

# Check if the optimization was successful
if model.status == gb.GRB.OPTIMAL:
    # Get the optimal solution and objective value
    optimal_solution = {(month, term): X[month, term].x for month in months for term in term_structures}
    optimal_objective_value = model.objVal

    # Print the results
    print("Optimal Solution:")
    for key, value in optimal_solution.items():
        print(f"{key}: {value}")

    print("Optimal Objective Value:")
    print(f"Total Repayment: {optimal_objective_value}")

else:
    print("No feasible solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 12 columns and 48 nonzeros
Model fingerprint: 0x006e2a8b
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 4e+05]
Presolve removed 7 rows and 10 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.250000e+03   0.000000e+00      0s
       1    4.2735000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.273500000e+04
Optimal Solution:
('May', 1): 0.0
('May', 2): 0.0
('May', 3): 0.0
('June', 1): 0.0
('June', 2): 0.0
('June', 3): 0.0
('July', 1): 42000.0
('July', 2): 0.0
('July', 3): 0.0
('August', 

In [4]:
# Assuming the model has been optimized successfully

# Counting the number of investments made
num_investments = sum(1 for month, term in X.keys() if X[month, term].x > 0)

# Print the result
print(f"Number of different investments made: {num_investments}")


Number of different investments made: 1


In [5]:
# Assuming the decision variables (X) and the model are already defined

# Minimum required cash balance at the end of June
min_cash_balance_june = 20000

# Cash balance constraint at the end of June
model.addConstr(
    140000 + gb.quicksum(X['May', term] for term in term_structures) + gb.quicksum(X['June', term] for term in term_structures) >= min_cash_balance_june,
    "Cash_Balance_June"
)


<gurobi.Constr *Awaiting Model Update*>

In [6]:
# Assuming the decision variables (X) and the model are already defined

# Linear ratio constraint for cash balance at the end of July
model.addConstr(
    140000 + gb.quicksum(X['May', term] for term in term_structures) + gb.quicksum(X['June', term] for term in term_structures) + gb.quicksum(X['July', term] for term in term_structures) >= 0.65 * (140000 + gb.quicksum(X['May', term] for term in term_structures) + gb.quicksum(X['June', term] for term in term_structures)),
    "Cash_Balance_End_July"
)


<gurobi.Constr *Awaiting Model Update*>

In [7]:
# Assuming the decision variables (X) and the model are already defined

# Calculate total repayment
total_repayment = sum(X[i, j].x + X[i, j].x * interest_rate for i in months for j, interest_rate in zip(term_structures, [0.0175, 0.0225, 0.0275]))

# Print the result
print(f"Total Repayment: {total_repayment}")


Total Repayment: 42735.0


In [8]:
# Assuming the decision variables (X) and the model are already defined

# Calculate the amount withdrawn in May
withdrawn_in_may = sum(X['May', j].x for j in term_structures)

# Print the result
print(f"Amount Withdrawn in May: {withdrawn_in_may}")


Amount Withdrawn in May: 0.0


In [9]:
# Assuming the decision variables (X) and the model are already defined

# Calculate the cash balance at the end of August
cash_balance_august = (
    140000
    + sum(X['May', j].x for j in term_structures)
    + sum(X['June', j].x for j in term_structures)
    + sum(X['July', j].x for j in term_structures)
    + sum(X['August', j].x for j in term_structures)
)

# Print the result
print(f"Cash Balance at the end of August: {cash_balance_august}")


Cash Balance at the end of August: 182000.0


In [10]:
# Assuming the decision variables (X) and the model are already defined

# Update the minimum cash balance for June
new_min_cash_balance_june = 27500

# Update the cash balance constraint for June
model.getConstrByName("Cash_Balance_June").rhs = new_min_cash_balance_june

# Re-optimize the model
model.optimize()

# Check if the optimization was successful
if model.status == gb.GRB.OPTIMAL:
    # Calculate the new total repayment
    new_total_repayment = sum(X[i, j].x + X[i, j].x * interest_rate for i in months for j, interest_rate in zip(term_structures, [0.0175, 0.0225, 0.0275]))

    # Print the result
    print(f"New Total Repayment with increased cash balance for June: {new_total_repayment}")
else:
    print("No feasible solution found.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 12 columns and 63 nonzeros
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+04, 4e+05]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2735000e+04   2.750000e+04   0.000000e+00      0s
       1    6.0922813e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  6.092281250e+04
New Total Repayment with increased cash balance for June: 60922.8125


In [11]:
import gurobipy as gb

# Create a new optimization model for the dual problem
dual_model = gb.Model("Sunnyshore Bay Dual Program")

# Dual variables
y = {}
months_dual = ['May', 'June', 'July', 'August']

for month in months_dual:
    y[month] = dual_model.addVar(lb=0, vtype=gb.GRB.CONTINUOUS, name=f"y_{month}")

# Set the objective function to maximize
dual_model.setObjective(25000 * y['May'] + 20000 * y['June'] + 35000 * y['July'] + 18000 * y['August'], gb.GRB.MAXIMIZE)

# Add constraints
dual_model.addConstr(y['May'] <= 0.0175, "Dual_Constraint_May")
dual_model.addConstr(y['May'] + y['June'] <= 0.0225, "Dual_Constraint_June")
dual_model.addConstr(y['May'] + y['June'] + y['July'] <= 0.0275, "Dual_Constraint_July")
dual_model.addConstr(y['May'] + y['June'] + y['July'] + y['August'] <= 0.0275, "Dual_Constraint_August")

# Optimize the dual model
dual_model.optimize()

# Check if the optimization was successful
if dual_model.status == gb.GRB.OPTIMAL:
    # Get the optimal solution and objective value for the dual problem
    optimal_dual_solution = {month: y[month].x for month in months_dual}
    optimal_dual_objective_value = dual_model.objVal

    # Print the results
    print("Optimal Dual Solution:")
    for key, value in optimal_dual_solution.items():
        print(f"{key}: {value}")

    print("Optimal Dual Objective Value:")
    print(f"Dual W = {optimal_dual_objective_value}")
else:
    print("No feasible solution found for the dual problem.")


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.3.0 23D56)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 4 columns and 10 nonzeros
Model fingerprint: 0x6bb89142
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 4e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-02, 3e-02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 3 rows, 4 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.6250000e+02   0.000000e+00   0.000000e+00      0s
       0    9.6250000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.625000000e+02
Optimal Dual Solution:
May: 0.0
June: 0.0
July: 0.0275
August: 0.0
Optimal Dual Objective Value:
Dual W = 962.5


The ease of solving the primal or dual model can depend on various factors, including the specific characteristics of the problem, the structure of the constraints and objective functions, and the solver algorithms.

In general, both primal and dual formulations have their advantages and may be easier to solve in different situations:

Primal Model:

Typically, primal models are easier to interpret, and the decision variables directly correspond to the quantities of interest.
When the primal problem has a simple structure, solving it directly may be straightforward.
Dual Model:

The dual model can sometimes provide additional insights into the problem, such as shadow prices associated with constraints.
If the dual problem has a simpler structure or exhibits some favorable properties, it may be easier to solve.
The choice between solving the primal or dual model often depends on the specific goals of the analysis and the characteristics of the problem at hand. In practice, it is not uncommon to solve both the primal and dual problems to gain a more comprehensive understanding of the optimization problem.

In the context of linear programming problems like Sunnyshore Bay, Gurobi and other solvers are designed to handle both primal and dual formulations efficiently. The ease of solving either model may depend on the specific characteristics of the instance of the problem you are dealing with. It's recommended to experiment with both formulations and assess the performance based on your specific scenario.